# Imports

In [ ]:
# If anything isn't working.  Try reinstalling this version of sqlalchemy 
# and restarting the kernel by uncommenting the line below and running the cell.

# !pip install sqlalchemy==1.4

In [1]:
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import Column, String, Integer
from sqlalchemy import create_engine

In [2]:
import pandas as pd

# Declare the Schema of the User Table

In [3]:
Base = declarative_base()

In [4]:
class User(Base):
    
    __tablename__ = "users"
    
    id = Column(Integer(), primary_key = True)
    
    username = Column(String(30), unique = True, nullable = False)
    
    email = Column(String(85), unique = True, nullable = False)
    
    # This allows us to print out what it is.
    def __repr__(self):
        return f"id: {self.id}, username: {self.username}, email: {self.email}"
        
User()    

id: None, username: None, email: None

# Connect to a Database Type (SQLite, Postgres, etc.)

In [5]:
connection_string = "sqlite:///users.db"

engine = create_engine(connection_string, echo = True)  # Echo = log and show results.
engine

Engine(sqlite:///users.db)

# Convert the Python Class to a SQL Table

In [6]:
Base.metadata.create_all(engine)

2021-10-20 18:58:26,616 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-20 18:58:26,628 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-10-20 18:58:26,629 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-20 18:58:26,631 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2021-10-20 18:58:26,633 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-20 18:58:26,640 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	username VARCHAR(30) NOT NULL, 
	email VARCHAR(85) NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (username), 
	UNIQUE (email)
)


2021-10-20 18:58:26,641 INFO sqlalchemy.engine.Engine [no key 0.00166s] ()
2021-10-20 18:58:26,649 INFO sqlalchemy.engine.Engine COMMIT


# Add User Entries with a Session

In [7]:
Session = sessionmaker()
Session

sessionmaker(class_='Session', bind=None, autoflush=True, autocommit=False, expire_on_commit=True)

In [8]:
local_session = Session(bind = engine)
local_session

In [9]:
new_user = User(id = 1, username = "Tommy_Pancakes", email = "tom@leethacker.com")
new_user

id: 1, username: Tommy_Pancakes, email: tom@leethacker.com

In [10]:
new_user2 = User(id = 2, username = "Nicky_Waffles", email = "nick@42.com")
new_user2

id: 2, username: Nicky_Waffles, email: nick@42.com

In [11]:
local_session.add(new_user)
local_session.add(new_user2)

local_session.commit()

2021-10-20 18:58:36,202 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-20 18:58:36,211 INFO sqlalchemy.engine.Engine INSERT INTO users (id, username, email) VALUES (?, ?, ?)
2021-10-20 18:58:36,212 INFO sqlalchemy.engine.Engine [generated in 0.00199s] ((1, 'Tommy_Pancakes', 'tom@leethacker.com'), (2, 'Nicky_Waffles', 'nick@42.com'))
2021-10-20 18:58:36,219 INFO sqlalchemy.engine.Engine COMMIT


# Use a Session to Perform Queries

### Pythonic Style SQL Queries with local_session.query()

In [ ]:
# Neat line of code Jupyter doesn't like.

# query_result = local_session.query(User.username) \    # Like SELECT
#                             .filter(User.id == 1) \    # Like WHERE
#                             .order_by(User.username) \ # Like ORDER BY
#                             .first()                   # Like LIMIT 1
# query_result

In [12]:
#Ugly line of same code above.

query_result = local_session.query(User.username).filter(User.id == 1).order_by(User.username).first()  # Like LIMIT 1
query_result

2021-10-20 19:10:13,792 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-20 19:10:13,805 INFO sqlalchemy.engine.Engine SELECT users.username AS users_username 
FROM users 
WHERE users.id = ? ORDER BY users.username
 LIMIT ? OFFSET ?
2021-10-20 19:10:13,805 INFO sqlalchemy.engine.Engine [generated in 0.00124s] (1, 1, 0)


('Tommy_Pancakes',)

### You can use straight up SQL queries too with engine.execute()

In [13]:
SQL_STRING = "SELECT * FROM Users WHERE id == 1"

data = engine.execute(SQL_STRING)

for record in data:
    print(record)

2021-10-20 19:10:56,828 INFO sqlalchemy.engine.Engine SELECT * FROM Users WHERE id == 1
2021-10-20 19:10:56,829 INFO sqlalchemy.engine.Engine [raw sql] ()
(1, 'Tommy_Pancakes', 'tom@leethacker.com')


### Lastly, you can use pandas

# Introspect with Inspector

In [14]:
from sqlalchemy import inspect  # Students, put this at the top, just for ease in teaching.
inspector = inspect(engine)
inspector

In [15]:
dir(inspector)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_connectable_insp',
 '_connection_insp',
 '_construct',
 '_engine_insp',
 '_index_sort_exprs',
 '_init_connection',
 '_init_engine',
 '_init_legacy',
 '_inspection_context',
 '_op_context_requires_connect',
 '_operation_context',
 '_reflect_check_constraints',
 '_reflect_col_sequence',
 '_reflect_column',
 '_reflect_fk',
 '_reflect_indexes',
 '_reflect_pk',
 '_reflect_table_comment',
 '_reflect_unique_constraints',
 'bind',
 'default_schema_name',
 'dialect',
 'engine',
 'from_engine',
 'get_check_constraints',
 'get_columns',
 'get_foreign_keys',
 'get_indexes',
 'get_pk_constraint',
 'get_schema_names',
 'get_sequence_

In [16]:
columns = inspector.get_columns('users')
columns

2021-10-20 19:12:33,995 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("users")
2021-10-20 19:12:33,996 INFO sqlalchemy.engine.Engine [raw sql] ()


[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'username',
  'type': VARCHAR(length=30),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'email',
  'type': VARCHAR(length=85),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

## Loop over the columns and print them out

In [17]:
for column in columns:
    print(column['name'])


id
username
email


## Read SQL Database into Dataframe

In [18]:
sample_query = "SELECT * FROM Users"
df = pd.read_sql(sample_query, engine)
df.head()

2021-10-20 19:13:35,158 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM Users")
2021-10-20 19:13:35,159 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-20 19:13:35,162 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM Users")
2021-10-20 19:13:35,163 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-20 19:13:35,168 INFO sqlalchemy.engine.Engine SELECT * FROM Users
2021-10-20 19:13:35,169 INFO sqlalchemy.engine.Engine [raw sql] ()


,id,username,email
0,1,Tommy_Pancakes,tom@leethacker.com
1,2,Nicky_Waffles,nick@42.com
